# Human Activity Recognition Using Acceleration Data From Cell Phones

## 1. Business Understanding

## 2. Explaratory Data Analysis

## 3. Machine Learning Model

## 4. Results and Recommendations

## 5. Limitations and Next Steps